In [1]:
import os
import pickle
import numpy as np
import pandas as pd

In [2]:
def atisfold(fold):
    assert fold in range(5)
    filename = os.path.join('data','atis.fold'+str(fold)+'.pkl')
    f = open(filename, 'rb')
    try:
        train_set, valid_set, test_set, dicts = pickle.load(f, encoding='latin1')
    except:
        train_set, valid_set, test_set, dicts = pickle.load(f)
    return train_set, valid_set, test_set, dicts

In [3]:
train_set, valid_set, test_set, dic = atisfold(0) 

In [4]:
def rev_map(d):
    return dict((d[i],i) for i in d)

labels2idx = rev_map(dic['labels2idx'])
tables2idx = rev_map(dic['tables2idx'])
words2idx = rev_map(dic['words2idx'])

word_indices = train_set[0]
name_entities = train_set[1]
labels = train_set[2]

print ("Number of sentences : ",len(word_indices))

def display(n):
    sense = []
    for i in range(len(word_indices[n])):
    #     sense.append({"word_index":word_indices[0][i],"word":words2idx[word_indices[0][i]],"entity_index":name_entities[0][i],"entity":tables2idx[name_entities[0][i]],"label_index":labels[0][i],"label":labels2idx[labels[0][i]]})
        sense.append({"word":words2idx[word_indices[n][i]],"entity":tables2idx[name_entities[n][i]],"label":labels2idx[labels[n][i]]})
    return pd.DataFrame(sense)

('Number of sentences : ', 3983)


In [5]:
display(2)

,entity,label,word
0,<NOTABLE>,O,i
1,<NOTABLE>,O,'d
2,<NOTABLE>,O,like
3,<NOTABLE>,O,to
4,<NOTABLE>,O,know
5,<NOTABLE>,O,the
6,B-flight_mod,B-flight_mod,earliest
7,<NOTABLE>,O,flight
8,<NOTABLE>,O,from
9,B-city_name,B-fromloc.city_name,boston


# Reading IOB files from data2/

In [2]:
def get_data(filename):
    df = pd.read_csv(filename,delim_whitespace=True,names=['word','label'])
    beg_indices = list(df[df['word'] == 'BOS'].index)+[df.shape[0]]
    sents,labels,intents = [],[],[]
    for i in range(len(beg_indices[:-1])):
        sents.append(df[beg_indices[i]+1:beg_indices[i+1]-1]['word'].values)
        labels.append(df[beg_indices[i]+1:beg_indices[i+1]-1]['label'].values)
        intents.append(df.loc[beg_indices[i+1]-1]['label'])    
    return np.array(sents),np.array(labels),np.array(intents)

def get_data2(filename):
    with open(filename) as f:
        contents = f.read()
    sents,labels,intents = [],[],[]
    for line in contents.strip().split('\n'):
        words,labs = [i.split(' ') for i in line.split('\t')]
        sents.append(words[1:-1])
        labels.append(labs[1:-1])
        intents.append(labs[-1])
    return np.array(sents),np.array(labels),np.array(intents)

read_method = {'data2/atis-2.dev.w-intent.iob':get_data,
               'data2/atis.train.w-intent.iob':get_data2,
               'data2/atis.test.w-intent.iob':get_data,
              'data2/atis-2.train.w-intent.iob':get_data2}

def fetch_data(fname):
    func = read_method[fname]
    return func(fname)

In [6]:
# Example
sents,labels,intents = fetch_data('data2/atis.train.w-intent.iob')
def display(n,intents):
    sense = []
    print ("INTENT : ",intents[n])
    for i in range(len(sents[n])):
    #     sense.append({"word_index":word_indices[0][i],"word":words2idx[word_indices[0][i]],"entity_index":name_entities[0][i],"entity":tables2idx[name_entities[0][i]],"label_index":labels[0][i],"label":labels2idx[labels[0][i]]})
        sense.append({"word":sents[n][i],"label":labels[n][i]})
    return pd.DataFrame(sense)

display(1)

INTENT :  atis_flight


,label,word
0,O,what
1,O,flights
2,O,are
3,O,available
4,O,from
5,B-fromloc.city_name,pittsburgh
6,O,to
7,B-toloc.city_name,baltimore
8,O,on
9,B-depart_date.day_name,thursday
